# Predicting Sales - Data Preparation

Since Salesforce has a limit on the number of columns that can be used in a report, I wrote a custom SFDC script to consolidate the other columns I needed into a single field that was encoded in JSON.

This Python Notebook takes a raw CSV file generated by Salesforce and prepares it for analysis and use in a Machine Learning model.

#### Import dependencies

In [1]:
# Define some exclusions for PEP8 that don't apply when the Jupyter Notebook
#   is exported to .py file
# pylint: disable=pointless-statement
# pylint: disable=fixme
# pylint: disable=expression-not-assigned
# pylint: disable=missing-module-docstring
# pylint: disable=invalid-name

import os
import json
# import sys
# import re
from math import isnan
from collections import Counter

import pandas as pd
# from pandas._libs.tslibs.parsing import DateParseError
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

sns.set_theme()

### Load the data into a Pandas dataframe
Define the path to the dataset file
Define the name of the label column

In [2]:
rootdir = os.getcwd()
infile = os.path.join(rootdir, 'data', 'dummy_sfdc_data.csv')
df = pd.read_csv(infile)
df.dtypes

Opportunity ID                              object
Annual Recurring Revenue (ARR) Currency     object
Annual Recurring Revenue (ARR)             float64
array_of_sfdc_formulas                      object
Team Territory Group                        object
Age                                        float64
Push Count                                   int64
Opportunity Owner                           object
Industry                                    object
Won                                          int64
dtype: object

#### Customized variables for this dataset

In [3]:
# can be either "train" or "predict"
DATA_PREP_MODE = 'train'

LABEL_COLUMN_NAME = "Won"
# NORMALIZE_METHOD = "min_max"

FEATURE_VALID_RANGES = {
    'Age': {'min': 20, 'max': 720},
    'Annual Recurring Revenue (ARR)': {'min': 2000, 'max': 2000000},
    'quarter_closed': {'min': 1, 'max': 4},
    'quarter_created': {'min': 1, 'max': 4},
}

FEATURE_PROPER_DATATYPES = {"Annual Recurring Revenue (ARR)": "int64",
               "Age": "int64",
               'partner_involved': "bool"
               }

FEATURES_TO_OHE = ['Team Territory Group', 'Opportunity Owner',
                         'Industry', 'primary_product']

# def get_stat(col_name, stat_name):
#     """docstring TBD"""
    # return df.describe(include="all").loc[stat_name].loc[col_name]

# Finding the percentiles:
# def find_nearest_index(array, value):
#     """docstring TBD"""
#     array = np.asarray(array)
#     idx = (np.abs(array - value)).argmin()
#     return idx

# validate configuration above.
assert DATA_PREP_MODE in ['train', 'predict'], \
    "DATA_PREP_MODE must be either train or predict. Exiting"
    
if DATA_PREP_MODE == 'train':
    FEATURE_PROPER_DATATYPES[LABEL_COLUMN_NAME] = 'bool'
    # print(FEATURE_PROPER_DATATYPES)

df.head(10)

,Opportunity ID,Annual Recurring Revenue (ARR) Currency,Annual Recurring Revenue (ARR),array_of_sfdc_formulas,Team Territory Group,Age,Push Count,Opportunity Owner,Industry,Won
0,abcdefgh1234567,USD,1420.69,"{ ""quarter_created"": 1, ""quarter_closed"": 3, ""...",South West Commercial,123.0,1,Jim Halpert,Healthcare,0
1,abcdefgh1234568,USD,1234.00,"{ ""quarter_created"": 2, ""quarter_closed"": 3, ""...",North East Commercial,87.0,2,Dwight Schrute,Finance,1
2,abcdefgh1234569,USD,6666.00,"{ ""quarter_created"": 3, ""quarter_closed"": 4, ""...",South East Commercial,100.0,3,Phyllis Vance,Services,1
3,abcdefgh1234570,USD,69420.69,"{ ""quarter_created"": 4, ""quarter_closed"": 4, ""...",South Central Commercial,364.0,2,Stanley Hudson,Manufacturing,0


## Correcting/converting the feature data types

#### Extracting the Salesforce JSON fields into individual fields
Salesforce has a strict limitation on the number of columns it can export in a report. One way around this is to combine several fields into a single one using JSON.

In [4]:
# Establishing invalid default values to create the features
df['quarter_created'] = 0
df['quarter_closed'] = 0
df['primary_product'] = None
df['partner_involved'] = None

# convert JSON to new columns
for index_iter in df.index:
    fields_as_json_str = df['array_of_sfdc_formulas'][index_iter]
    fields_as_dict = json.loads(fields_as_json_str)
    for colname, value in fields_as_dict.items():
        df.loc[df.index == index_iter, colname] = value


for colname, newdatatype in FEATURE_PROPER_DATATYPES.items():
    if DATA_PREP_MODE == "train" or (DATA_PREP_MODE == "predict" and colname != LABEL_COLUMN_NAME):
        df[colname] = df[colname].astype(newdatatype)

# df.dtypes

## Drop the unused features

In [5]:
df.drop(
    columns=[
        "Annual Recurring Revenue (ARR) Currency",
        "Opportunity ID",     # leave this is for easy re-joining the data
        "array_of_sfdc_formulas"
    ],
    inplace=True,
)

# df.dtypes

## Correcting salesforce user error and other non-use cases

Removes the following:
* opps with $0 or less ARR - these are refunds, free upgrades, etc.
* opps with an age of less than 14 days - these were likely duplicates or multiple opps made for quoting purposes
* 

In [6]:
for colname, ranges_dict in FEATURE_VALID_RANGES.items():
    min_value = ranges_dict['min']
    max_value = ranges_dict['max']
    indexes_to_drop = df[ (df[colname] < min_value) | (df[colname] > max_value) ].index
    df.drop(indexes_to_drop, inplace=True)

## Winsorizing numerical outliers
Disabled for now

In [7]:
# # Winsorize the top 1% and bottom 1%
# percentile = 0.01

# for iter_column_name in df.select_dtypes(include=np.number).columns.tolist():
#     new_column_name = iter_column_name + "_winsorized"

#     winsorized_data = stats.mstats.winsorize(
#         df[iter_column_name], limits=[percentile, percentile], inplace=False
#     )

#     if (winsorized_data == df[iter_column_name]).all():
#         print(
#             f"Winsorization on column {iter_column_name} had no effect. Not changing this column."
#         )
#         continue

#     df[new_column_name] = winsorized_data
#     df.drop(columns=iter_column_name, inplace=True)
#     print(
#         f"Winsorized column {iter_column_name} to {new_column_name} and removed original column."
#     )

## Normalizing numerical ranges
Disabled for now


In [8]:
# def normalize(df_local, column_name, normalize_method_name):
#     """docstring TBD"""
#     df_temp = df_local.copy()
#     new_columnname = column_name + "_normalized"

#     if normalize_method_name == "absolute_range":
#         df_temp[new_columnname] = (
#             df_temp[column_name] / df_temp[column_name].abs().max()
#         )

#     elif normalize_method_name == "min_max":
#         # rescales a features to be in the range [0,1]
#         df_temp[new_columnname] = (
#             df_temp[column_name] - df_temp[column_name].min()
#         ) / (df_temp[column_name].max() - df_temp[column_name].min())

#     elif normalize_method_name == "z_score":
#         df_temp[new_columnname] = (
#             df_temp[column_name] - df_temp[column_name].mean()
#         ) / df_temp[column_name].std()

#     else:
#         raise NameError("Unrecogized normalization method")

#     df_temp.drop(columns=column_name, inplace=True)
#     print(
#         f"Normalized column {column_name} into {new_columnname} using {normalize_method_name}. Removed original."
#     )
#     return df_temp


# # iterate through the list of current numeric columns
# for iter_column_name in df.select_dtypes(include=np.number).columns.tolist():
#     df = normalize(df, iter_column_name, NORMALIZE_METHOD)

# One hot encoding strings
Convert strings like sales rep names, industries, and product names to machine friendly columns.

In [9]:
for iter_column_name in FEATURES_TO_OHE:
    # define a new column names, it automatically adds a _ to the end of the prefix
    new_column_prefix = iter_column_name

    # create a one-hot encoded version in a new dataframe
    temp_df = pd.get_dummies(df[iter_column_name], prefix=new_column_prefix)

    # merge the new dataframe into the existing one
    df = df.join(temp_df)

    # remove the original column now that it has been encoded 
    # into the existing dataframe
    df.drop(columns=iter_column_name, inplace=True)
    
    print(f'One-hot encoded: {iter_column_name} into {new_column_prefix}*')
    
df.dtypes

One-hot encoded: Team Territory Group into Team Territory Group*
One-hot encoded: Opportunity Owner into Opportunity Owner*
One-hot encoded: Industry into Industry*
One-hot encoded: primary_product into primary_product*


Annual Recurring Revenue (ARR)                   int64
Age                                              int64
Push Count                                       int64
Won                                               bool
quarter_created                                  int64
quarter_closed                                   int64
partner_involved                                  bool
Team Territory Group_South Central Commercial     bool
Team Territory Group_South East Commercial        bool
Opportunity Owner_Phyllis Vance                   bool
Opportunity Owner_Stanley Hudson                  bool
Industry_Manufacturing                            bool
Industry_Services                                 bool
primary_product_Product3                          bool
primary_product_Product4                          bool
dtype: object

# Re-order the columns
Sort the column names alphabetically, but make sure the 'label' column is always last.

In [10]:
# alphabetically sort the column names, but leave the label as the last column
# since the label will be dropped for predictive datasets
column_order = sorted(df.columns)

if DATA_PREP_MODE == "train":
    column_order.remove(LABEL_COLUMN_NAME)
    column_order.append(LABEL_COLUMN_NAME)

df = df.reindex(column_order, axis=1)

# Final tests

In [11]:
# check for missing values
# check for any remaining strings
df.describe(include="all")

,Age,Annual Recurring Revenue (ARR),Industry_Manufacturing,Industry_Services,Opportunity Owner_Phyllis Vance,Opportunity Owner_Stanley Hudson,Push Count,Team Territory Group_South Central Commercial,Team Territory Group_South East Commercial,partner_involved,primary_product_Product3,primary_product_Product4,quarter_closed,quarter_created,Won
count,2.00000,2.000000,2,2,2,2,2.000000,2,2,2,2,2,2.0,2.000000,2
unique,NaN,NaN,2,2,2,2,NaN,2,2,1,2,2,NaN,NaN,2
top,NaN,NaN,False,True,True,False,NaN,False,True,True,True,False,NaN,NaN,True
freq,NaN,NaN,1,1,1,1,NaN,1,1,2,1,1,NaN,NaN,1
mean,232.00000,38043.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,NaN,4.0,3.500000,NaN
std,186.67619,44373.778947,NaN,NaN,NaN,NaN,0.707107,NaN,NaN,NaN,NaN,NaN,0.0,0.707107,NaN
min,100.00000,6666.000000,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,4.0,3.000000,NaN
25%,166.00000,22354.500000,NaN,NaN,NaN,NaN,2.250000,NaN,NaN,NaN,NaN,NaN,4.0,3.250000,NaN
50%,232.00000,38043.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,NaN,4.0,3.500000,NaN
75%,298.00000,53731.500000,NaN,NaN,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,NaN,4.0,3.750000,NaN


In [12]:
# show the final datatypes before exporting to CSV
df.dtypes

Age                                              int64
Annual Recurring Revenue (ARR)                   int64
Industry_Manufacturing                            bool
Industry_Services                                 bool
Opportunity Owner_Phyllis Vance                   bool
Opportunity Owner_Stanley Hudson                  bool
Push Count                                       int64
Team Territory Group_South Central Commercial     bool
Team Territory Group_South East Commercial        bool
partner_involved                                  bool
primary_product_Product3                          bool
primary_product_Product4                          bool
quarter_closed                                   int64
quarter_created                                  int64
Won                                               bool
dtype: object

In [13]:
df.head()

,Age,Annual Recurring Revenue (ARR),Industry_Manufacturing,Industry_Services,Opportunity Owner_Phyllis Vance,Opportunity Owner_Stanley Hudson,Push Count,Team Territory Group_South Central Commercial,Team Territory Group_South East Commercial,partner_involved,primary_product_Product3,primary_product_Product4,quarter_closed,quarter_created,Won
2,100,6666,False,True,True,False,3,False,True,True,True,False,4,3,True
3,364,69420,True,False,False,True,2,True,False,True,False,True,4,4,False


In [14]:
#TODO: check the % of labels that are True vs False, ensure balance

# Basic data validation before modeling

Check ranges, values, datatypes, missing values, 

In [15]:
for colname, ranges_dict in FEATURE_VALID_RANGES.items():
    min_value = ranges_dict['min']
    max_value = ranges_dict['max']
    for index, row in df.iterrows():
        val = df[colname][index]
        assert min_value <= val <= max_value, f'Out of range: {colname} value of {val} is not between {min_value} and {max_value}'
        
print("all data tests passed successfully.")

# df.head(3)
df.dtypes

all data tests passed successfully.


Age                                              int64
Annual Recurring Revenue (ARR)                   int64
Industry_Manufacturing                            bool
Industry_Services                                 bool
Opportunity Owner_Phyllis Vance                   bool
Opportunity Owner_Stanley Hudson                  bool
Push Count                                       int64
Team Territory Group_South Central Commercial     bool
Team Territory Group_South East Commercial        bool
partner_involved                                  bool
primary_product_Product3                          bool
primary_product_Product4                          bool
quarter_closed                                   int64
quarter_created                                  int64
Won                                               bool
dtype: object

# Store prepped data in new CSV
Makes it easy to run in separate notebooks.

In [16]:
outfile = os.path.splitext(infile)[0] + "_" + DATA_PREP_MODE + ".csv"
df.to_csv(outfile, index=False)
print(f"{DATA_PREP_MODE} data saved to new CSV file:\n{outfile}")

train data saved to new CSV file:
/Users/the-molecular-man/source_code/machine-learning/sales_modeling/data/dummy_sfdc_data_train.csv
